In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [4]:

query = """
    SELECT  p.display_name as Name,
            a.artifact_name as Repo,
            m.metric_name as Metric,
            ts.sample_date as Date,
            ts.amount as Value
    FROM metrics_v0 m
    JOIN timeseries_metrics_by_artifact_v0 ts
    on ts.metric_id = m.metric_id
    JOIN artifacts_by_project_v1 a
    on a.artifact_id = ts.artifact_id
    JOIN projects_v1 p
    on p.project_id = a.project_id
    where metric_name = 'GITHUB_active_developers_monthly'
    and a.artifact_source = 'GITHUB'
    and YEAR(ts.sample_date) = 2024
    and p.project_name IN (
        '9lives-fluidity-money',
        'angel',
        'arbos-foundry-bernard-wagner',
        'cobuilders-xyz',
        'codetracer-metacraft-labs',
        'ember-shahankhatch',
        'enclave-gnosisguild',
        'fairblock',
        'fiet-protocol-usherlabs',
        'nuffle-labs',
        'opensource-observer',
        'passportxyz',
        'pyth-network',
        'rather_labs',
        'redstone-finance',
        'remix-arbitrum-welldonestudio',
        'runtimeverification',
        'solang-hyperledger-solang',
        'stylusport-teamscanworks',
        'sylow-warlock-labs',
        'syndicateprotocol',
        'third-web',
        'trailofbits',
        'wakeuplabs',
        'walnuthq')
"""

df = client.to_pandas(query)
df

,Name,Repo,Metric,Date,Value
0,Trail of Bits Security Reviews,sigstore-rekor-types,GITHUB_active_developers_monthly,2024-07-01,1
1,Debid - Fairblock,sharegenerationclient,GITHUB_active_developers_monthly,2024-07-01,1
2,Trail of Bits Security Reviews,awesome-ml-security,GITHUB_active_developers_monthly,2024-07-01,1
3,Open Source Observer,oso,GITHUB_active_developers_monthly,2024-07-01,5
4,Walnut,cairovm.codes,GITHUB_active_developers_monthly,2024-07-01,1
...,...,...,...,...,...
2385,Walnut,starknet-foundry,GITHUB_active_developers_monthly,2024-06-01,1
2386,Trail of Bits Security Reviews,debloater-eval,GITHUB_active_developers_monthly,2024-06-01,1
2387,Trail of Bits Security Reviews,zkdocs,GITHUB_active_developers_monthly,2024-06-01,1
2388,Runtime Verification,kontrol-solady,GITHUB_active_developers_monthly,2024-06-01,1


In [ ]:
import plotly.express as px

# Choose your project
project_name = 'Open Source Observer'  # replace with your project
project_data = df[df['Name'].str.contains(project_name, case=False)].copy()

# First sort the data by date
project_data['Date'] = pd.to_datetime(project_data['Date'])
project_data = project_data.sort_values('Date')

# Create the line plot
fig = px.line(project_data, 
              x='Date', 
              y='Value',
              color='Repo',  # This will create separate lines for each repo
              title=f'Monthly Active Developers Trend for {project_name}',
              labels={'Value': 'Number of Active Developers',
                     'Date': 'Month',
                     'Repo': 'Repository'})

# Customize the layout
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Active Developers",
    legend_title="Repository",
    hovermode='x unified'
)

# Show the plot
fig.show()